In [6]:

# coding: utf-8

# In[44]:


import os
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


# In[3]:


os.chdir("C:\Users\User\Downloads\data_set\Data Sets")


# In[20]:


data=pd.read_csv("train.csv",index_col=0)


# In[26]:


comments=np.array(data.iloc[:,[2]]).tolist()


# In[39]:


##################################################### cleaning using regex ##################################################

clean=[]
for i in comments:
    for j in i:
        a=re.sub(r"[^\x00-\x7f]"," ",j)
        a=a.lower()
        a=re.sub(r"(<br />)"," ",a)
        a=re.sub(r'(://)',' ', a)
        
        a=a.strip()
        clean.append(a)
clean


# In[59]:


################################################ tokenization and lemmatization and punctuation removal ######################################
tokenized_words=[]
for i in clean:
    tokenized_words.append(word_tokenize(i))

tokenized_words



# In[60]:


vocabulary=[]

lemmatizer=WordNetLemmatizer()

corpus=[]

for i in tokenized_words:
    temp=""
    for j in i:
        if j.isalpha() and j not in vocabulary and j not in stopwords.words("english"):
            lemmatizer.lemmatize(j)
            vocabulary.append(j)
        if j.isalpha() and j not in stopwords.words("english"):
            temp+=j+" "
    corpus.append(word_tokenize(temp.strip()))


word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}

idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

corpus




[['huh', 'anyway', 'check', 'tube', 'channel'],
 ['shaking', 'sexy', 'ass', 'channel', 'enjoy'],
 ['watch', 'check'],
 ['subscribe', 'channel'],
 ['u', 'check', 'channel', 'tell', 'next'],
 ['started',
  'reading',
  'stop',
  'subscribe',
  'within',
  'one',
  'day',
  'entire',
  'family',
  'die',
  'want',
  'stay',
  'alive',
  'subscribe',
  'right'],
 ['https'],
 ['please', 'like', 'https'],
 ['hello',
  'like',
  'gaming',
  'art',
  'videos',
  'scientific',
  'experiments',
  'tutorials',
  'lyrics',
  'videos',
  'much',
  'much',
  'please',
  'check',
  'channel',
  'subscribe',
  'started',
  'soon',
  'hope',
  'able',
  'cover',
  'expectations',
  'also',
  'check',
  'got',
  'far'],
 ['checking', 'views'],
 ['show', 'auburn', 'pride', 'http'],
 ['check', 'channel'],
 ['hey',
  'guys',
  'im',
  'yr',
  'old',
  'music',
  'producer',
  'make',
  'chiptunes',
  'music',
  'would',
  'wonderful',
  'checked',
  'remixes',
  'even',
  'gangnamstyle',
  'remix',
  'woul

In [20]:
####################################################### word2vec #############################################################

window_size=2
idx_pairs=[]
word_pairs=[]
for i in corpus:
    indeces_of_each_word=[]
    for j in i:
        
        indeces_of_each_word.append(word2idx[j])
    
    for center_word in range(len(indeces_of_each_word)):
        for w in range(-window_size,window_size+1):
            position_of_context=center_word+w
            if position_of_context<0 or position_of_context>=len(indeces_of_each_word) or position_of_context==center_word:
                continue
            context_word_index=indeces_of_each_word[position_of_context]
            idx_pairs.append([indeces_of_each_word[center_word],context_word_index])
            word_pairs.append((idx2word[center_word],idx2word[context_word_index]))
idx_pairs=np.array(idx_pairs)
print idx_pairs

[['huh' 'anyway']
 ['huh' 'check']
 ['anyway' 'huh']
 ...
 ['anyway' 'wiredo']
 ['check' 'shakira']
 ['check' 'u']]


In [40]:
########################################################### one-hot vectors #######################################

vocab_size=len(vocabulary)
temp=np.zeros(vocab_size)
temp

def to_one_hot(data_point_index, vocab_size):
    temp = np.zeros(vocab_size)
    temp[data_point_index] = 1
    return temp

x_train=[]
y_train=[]

for i in idx_pairs:
    x_train.append(to_one_hot(i[0],vocab_size))
    y_train.append(to_one_hot(i[1],vocab_size))

x_train=np.asarray(x_train)
y_train=np.asarray(y_train)
x_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [44]:
y_train

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [39]:
############################################################### model training #############################################

from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()
clf.fit(x_train,y_train)

ValueError: bad input shape (31214L, 2426L)

In [36]:
################################################### reading test data #################################################
test=pd.read_csv("test.csv",index_col=0)
test.columns
test_comments=test["CONTENT"].tolist()
#test_comments


tokenized_words_test=[]
for i in test_comments:
    tokenized_words_test.append(word_tokenize(i))

tokenized_words_test



# In[60]:


vocabulary_test=[]

lemmatizer=WordNetLemmatizer()

corpus_test=[]

for i in tokenized_words_test:
    temp=""
    for j in i:
        if j.isalpha() and j not in vocabulary and j not in stopwords.words("english"):
            lemmatizer.lemmatize(j)
            vocabulary.append(j)
        if j.isalpha() and j not in stopwords.words("english"):
            temp+=j+" "
    corpus_test.append(word_tokenize(temp.strip()))


word2idx_test = {w: idx for (idx, w) in enumerate(vocabulary)}

idx2word_test = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size_test = len(vocabulary)

corpus_test

UnicodeDecodeError: 'ascii' codec can't decode byte 0xef in position 4: ordinal not in range(128)